# 1. Setup

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))

In [ ]:
# Horizontal Scrolling
import io
import base64
from IPython.display import HTML, display

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.transforms as mt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import seaborn as sns
from pandas import DataFrame, Series, Timedelta
from scipy.ndimage import binary_dilation
from sklearn.metrics import mean_absolute_error as MAE

In [ ]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from xgboost import XGBRegressor

In [ ]:
from utils.loading import load_all_raw_data
from utils.preprocessing import process_all_dfs
from utils.merging import merge_all_dfs
from utils.feature_engineering import *

In [ ]:
pd.set_option(
    "display.float_format",
    lambda x: f"{x:.2e}" if abs(x) < 0.01 and x != 0 else f"{x:.2f}",
)
pd.set_option('display.max_columns', 100)
pd.set_option("display.max_rows", 100)

In [ ]:
RAW_DATA_PATH = "../data/raw_data/"
ADDITIONAL_DATA_PATH = "../data/additional_data/"

SEGMENT_C = ["county", "product_type", "is_business"]
CATEGORICAL_C = ["county", "product_type", "is_business", "is_consumption"]
TARGET_C = [
    "county",
    "product_type",
    "is_business",
    "is_consumption",
    "datetime",
]
RAND = 10

# 2. Processing

In [ ]:
processed_dfs = process_all_dfs(
    load_all_raw_data(RAW_DATA_PATH, ADDITIONAL_DATA_PATH)
)

In [ ]:
# processed_dfs.keys()

In [ ]:
df = merge_all_dfs(processed_dfs, how="left")

In [ ]:
df = add_dst_flag(df)

In [ ]:
df = add_cyclic_datetime_features(df, drop_raw=True)

In [ ]:
for lag in [2, 3, 7]:
# for lag in range(2, 15):
    df = df.merge(
        get_lag(processed_dfs["train"][TARGET_C + ["target"]], lag=lag),
        how="left",
        on=TARGET_C,
    )

In [ ]:
for window in [24, 24 * 3, 24 * 7, 24 * 14]:
    # for window in [24 * i for i in range(1, 15)]:
    df = df.merge(
        get_moving_average(
            processed_dfs["train"]
            .set_index("datetime")
            .sort_index()
            .groupby(CATEGORICAL_C, observed=True, as_index=False),
            columns=["target"],
            window=window,
            # ).dropna(),
        ),
        how="left",
        on=TARGET_C,
    )

In [ ]:
df["t_over_cap"] = (df["2d_lag_target"] / df["installed_capacity"]).astype(
    "float32"
)
df["t_over_eic"] = (df["2d_lag_target"] / df["eic_count"]).astype("float32")
df["cap_per_eic"] = (df["installed_capacity"] / df["eic_count"]).astype(
    "float32"
)

In [ ]:
# w, h = 20, 14

# fig, ax = plt.subplots(figsize=(w, h))
# # plt.figure(figsize=(w, h))

# sns.heatmap(
#     df.drop(
#         columns=df.select_dtypes(["category"]).columns.tolist()
#         + [
#             "datetime",
#             "data_block_id",
#             "date",
#         ]
#     ).corr(),
#     annot=True,
#     fmt=".1f",
#     annot_kws={"size": 7},
# )


# buf = io.BytesIO()
# fig.savefig(buf, format="png", bbox_inches="tight")
# buf.seek(0)
# img_base64 = base64.b64encode(buf.getvalue()).decode("utf-8")

# html_code = (
#     '<div style="overflow-x: auto; width: 100%;">'
#     '<img src="data:image/png;base64,{}" style="display: block; max-width: none; width: auto;">'
#     "</div>"
# ).format(img_base64)

# display(HTML(html_code))
# plt.close(fig)
# # plt.show()

In [ ]:
# df_for_missing = df[TARGET_C + ["target"]].dropna().copy()

# # Add hour index starting from the beginning
# df_for_missing["hour_index"] = (
#     (df_for_missing["datetime"] - df_for_missing["datetime"].min())
#     // pd.Timedelta(hours=1)
# ).astype(np.uint16)


# # Add a group index corresponding to the unique combination of all
# # categorical features (i.e. this feature is not equal to
# # prediction_unit_id): county, is_business, product_type,
# # is_consumption.
# # The maximum number of combinations is 16 * 4 * 2 * 2 = 256, but the
# # actual number of observed values is less than 256
# df_for_missing["group_index"] = (
#     df_for_missing.groupby(
#         CATEGORICAL_C,
#         observed=True,
#     )
#     .ngroup()
#     .astype(np.uint8)
# )

# # Create a 2D array with a shape equal to the number of groups and the
# # total number of hours between the minimum and maximum timestamps
# n_groups = df_for_missing["group_index"].nunique()
# n_hours = df_for_missing["hour_index"].max() + 1
# missmap = np.full((n_groups, n_hours), np.nan)

# # Fill the array with the corresponding flag values
# missmap[df_for_missing["group_index"], df_for_missing["hour_index"]] = (
#     df_for_missing["target"] != 0
# )

# # Dilate missing values because of high density of plot and isolated
# # missing values are not visible.
# n = 6  # How many hours before and after count as missing
# structure = np.ones((1, 2 * n + 1), dtype=bool)
# missing_dilated = binary_dilation(np.isnan(missmap), structure=structure)
# missmap[missing_dilated] = np.nan

# alpha = 0.8

# min_date = df_for_missing["datetime"].min()
# max_date = df_for_missing["datetime"].max()
# n_months = len(pd.date_range(start=min_date, end=max_date, freq="ME"))


# fig, ax = plt.subplots(figsize=(14, 7))
# sns.heatmap(
#     missmap,
#     cmap=sns.color_palette(["tab:blue", "tab:orange"]),
#     cbar=False,
#     alpha=alpha,
#     ax=ax,
# )

# xtick_indices = np.linspace(0, n_hours, num=n_months, dtype=int)
# xtick_labels = pd.date_range(min_date, max_date, freq="ME")


# ax.set_xticks(xtick_indices)
# ax.set_xticklabels(xtick_labels.strftime("%Y-%m"), rotation=45, ha="right")

# dx, dy = 0, -2.5
# for label in ax.get_yticklabels():
#     offset = mt.ScaledTranslation(dx / 72, dy / 72, fig.dpi_scale_trans)
#     label.set_transform(label.get_transform() + offset)

# legend_patches = [
#     mpatches.Patch(
#         facecolor="white",
#         label="Missing value",
#         edgecolor="black",
#         linewidth=0.5,
#     ),
#     mpatches.Patch(
#         facecolor="tab:blue",
#         alpha=alpha,
#         label="Zero value",
#         edgecolor="black",
#         linewidth=0.5,
#     ),
#     mpatches.Patch(
#         facecolor="tab:orange",
#         alpha=alpha,
#         label="Not zero value",
#         edgecolor="black",
#         linewidth=0.5,
#     ),
# ]

# plt.legend(
#     handles=legend_patches,
#     title="Data Presence",
#     title_fontsize=12,
#     bbox_to_anchor=(1, 1),
#     loc="upper left",
#     fontsize=11,
#     frameon=False,
# )

# plt.title(
#     "Heatmap of time series gaps for all combinations "
#     "of categorical features",
#     fontsize=13,
# )
# plt.xlabel("Month", fontsize=10)
# plt.ylabel("Group index", fontsize=10)
# plt.grid(False)
# plt.show()

# 3. Simple Baseline

In [ ]:
def split_by_equal_days(
    dt: Series,
    train_days: int = 1,
    fh_days: int = 1,  # Only daily predictions as per problem statement
    val_splits: int = 1,
    expanding: bool = True,
) -> list[dict[str, tuple[pd.Timestamp, pd.Timestamp]]]:
    dt = dt.dt.floor("D")  # Processing on a day scope

    train_days = Timedelta(days=train_days)
    train_days_delta = train_days - Timedelta(days=1)  # Indexing from 0

    fh_days = Timedelta(days=fh_days)
    fh_days_delta = fh_days - Timedelta(days=1)  # Indexing from 0

    first_day = dt.min()
    last_day = dt.max()

    # test_start = last_day - fh_days_delta
    # test_end = last_day + pd.Timedelta(hours=23)

    splits = []

    intermediate_period_start = first_day + train_days
    intermediate_period_end = last_day - fh_days
    intermediate_period_days = (
        intermediate_period_end - intermediate_period_start
    ).days
    base_step = Timedelta(days=intermediate_period_days // val_splits)
    step_rem = Timedelta(days=intermediate_period_days % val_splits)

    for step in range(val_splits):
        offset = base_step * step + min(Timedelta(days=step), step_rem)
        if expanding:
            train_start = first_day
            train_end = (
                train_start
                + train_days_delta
                + offset
                + pd.Timedelta(hours=23)
            )
        else:
            train_start = first_day + offset
            train_end = train_start + train_days_delta + pd.Timedelta(hours=23)

        val_start = train_end + pd.Timedelta(hours=1)
        val_end = val_start + fh_days_delta + pd.Timedelta(hours=23)
        splits.append(
            {
                "train": (train_start, train_end),
                "val": (val_start, val_end),
                # "test": (test_start, test_end),
            }
        )

    return splits

In [ ]:
print(
    f"First timestamp: {df["datetime"].min()}",
    f"Last timestamp: {df["datetime"].max()}",
    sep="\n"
)

First timestamp: 2021-09-01 00:00:00
Last timestamp: 2023-05-31 23:00:00


Since the minimum usable lag for prediction is 48 hours, this horizon is particularly well suited as a naive baseline.

In [ ]:
FEATURES_TO_DROP = ["datetime", "data_block_id", "date"]

In [ ]:
# Last day
test_end = df["datetime"].max()
test_start = test_end.normalize()  # 00 - 23
df_test = df.loc[df["datetime"] >= test_start].drop(FEATURES_TO_DROP, axis=1)

In [ ]:
# splits_expanding = split_by_equal_days(
#     dt=df["datetime"],
#     train_days=365,
#     fh_days=30,
#     val_splits=val_splits,
#     expanding=True,
# )

In [ ]:
# val_mae_2dlag = []

# for split in range(VAL_SPLITS):
#     val_df = df.loc[
#         (df["datetime"] >= splits_expanding[split]["val"][0])
#         & (df["datetime"] <= splits_expanding[split]["val"][1]),
#         ["target", "2d_lag_target"],
#     ]
#     val_mae_2dlag.append(MAE(val_df["target"], val_df["2d_lag_target"]))
# else:
#     test_df = df.loc[df["datetime"] >= test_start]
#     val_mae_2dlag.append(MAE(test_df["target"], test_df["2d_lag_target"]))

# print("Validation MAE:", np.round(val_mae_2dlag, 3))
# print("Validation mean MAE:", np.round(np.mean(val_mae_2dlag), 3))

In [ ]:
# df["datetime"].is_monotonic_increasing
# df = df.sort_values('datetime', ignore_index=True)

In [ ]:
VAL_SPLITS = 5

In [ ]:
(df["datetime"].min() + (df["datetime"].max() - df["datetime"].min()) * 0.8).normalize()

Timestamp('2023-01-24 00:00:00')

In [ ]:
df.iloc[round(df.index[-1] * 0.8)]["datetime"]

Timestamp('2023-01-25 00:00:00')

In [ ]:
splits_fixed = split_by_equal_days(
    dt=df["datetime"],
    train_days=365 + 30 * 3,
    fh_days=1,
    val_splits=VAL_SPLITS,
    expanding=False,
)
splits_fixed.extend(
    split_by_equal_days(
        dt=df["datetime"],
        train_days=365 + 30 * 6,
        fh_days=1,
        val_splits=VAL_SPLITS,
        expanding=False,
    )
)
# splits_fixed.extend(
#     split_by_equal_days(
#         dt=df["datetime"],
#         train_days=(
#             df.datetime.max() - df.datetime.min() - Timedelta(days=30)
#         ).days,  # all data except last 30 days
#         fh_days=1,
#         val_splits=VAL_SPLITS,
#         expanding=False,
#     )
# )

new_splits = []
for i, d in enumerate(splits_fixed):
    count = len(
        df.loc[
            (df["datetime"] >= d["val"][0]) & (df["datetime"] <= d["val"][1])
        ]
    )
    if count < 24:
        print(f"Split {i} only has {count} rows, removed")
    else:
        new_splits.append(d)

splits_fixed = new_splits
# for i, d in enumerate(splits_fixed):
#     print(i, "train", d["train"])
#     print(i, "valid", d["val"])

## Baselines comparing

In [ ]:
cat_cols = df.select_dtypes(include="category").columns.tolist()

xgb_mae = []
lgbm_mae = []
cb_mae = []

xgb_p = {
    "n_estimators": 100,
    "learning_rate": 0.1,
    "max_depth": 7,
    "random_state": RAND,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "objective": "reg:absoluteerror",
    "enable_categorical": True,
    "early_stopping_rounds": 20,
    "eval_metric": "mae",
    "n_jobs": -1,
}
lgbm_p = {
    "n_estimators": 100,
    "learning_rate": 0.1,
    "max_depth": 7,
    "random_state": RAND,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "objective": "regression_l1",
    "metric": "mae",
    "n_jobs": -1,
    "force_col_wise": True,
    "verbosity": -1,
}
cb_p = {
    "n_estimators": 100,
    "learning_rate": 0.1,
    "max_depth": 7,
    "random_state": RAND,
    "subsample": 0.8,
    "colsample_bylevel": 0.8,
    "loss_function": "MAE",
    "cat_features": cat_cols,
    "eval_metric": "MAE",
    "thread_count": -1,
    "verbose": False,
}

xgbr = XGBRegressor(**xgb_p)
lgbmr = LGBMRegressor(**lgbm_p)
cbr = CatBoostRegressor(**cb_p)

for model in [
    xgbr,
    lgbmr,
    cbr,
]:
    for i, split in enumerate(splits_fixed):
        df_train = df[
            (df["datetime"] >= split["train"][0])
            & (df["datetime"] <= split["train"][1])
        ].drop(FEATURES_TO_DROP, axis=1)
        df_val = df[
            (df["datetime"] >= split["val"][0])
            & (df["datetime"] <= split["val"][1])
        ].drop(FEATURES_TO_DROP, axis=1)

        X_train, y_train = (
            df_train.drop(["target"], axis=1),
            df_train["target"],
        )
        X_val, y_val = df_val.drop(["target"], axis=1), df_val["target"]
        # X_test, y_test = df_test.drop(["target"], axis=1), df_test["target"]

        eval_set = [
            (X_train, y_train),
            (X_val, y_val),
            # (X_test, y_test)
        ]

        # print(f"{i+1} split")
        # print("-" * 30)
        # print("train")
        # print("start:", split["train"][0])
        # print("end:", split["train"][1])
        # print("shape:", X_train.shape, y_train.shape)
        # print("-" * 30)
        # print("val")
        # print("start:", split["val"][0])
        # print("end:", split["val"][1])
        # print("shape:", X_val.shape, y_val.shape)
        # print("-" * 30)

        # print("test")
        # print("start:", test_start)
        # print("end:", test_end)
        # print("shape:", X_test.shape, y_test.shape)
        # print("-" * 30)

        if isinstance(model, XGBRegressor):
            # print("XGBRegressor")
            model.fit(
                X_train,
                y_train,
                eval_set=eval_set,
                verbose=0,
                # verbose=25,
            )
            xgb_mae.append(model.evals_result())

        elif isinstance(model, LGBMRegressor):
            # print("LGBMRegressor")
            model.fit(
                X_train,
                y_train,
                categorical_feature=cat_cols,
                eval_set=eval_set,
                callbacks=[
                    early_stopping(
                        stopping_rounds=20,
                        verbose=0,
                    ),
                    # log_evaluation(period=25),
                ],
            )
            lgbm_mae.append(model.evals_result_)

        elif isinstance(model, CatBoostRegressor):
            # print("CatBoostRegressor")
            model.fit(
                X_train, y_train, eval_set=eval_set, early_stopping_rounds=20
            )
            cb_mae.append(model.evals_result_)

In [ ]:
# Add model saving

In [ ]:
# xgb_mae[0]
# lgbm_mae[0]
# cb_mae[0]

In [ ]:
len(splits_fixed)

In [ ]:
total_splits = len(splits_fixed)
rows, cols = 2, VAL_SPLITS

fig, axs = plt.subplots(rows, cols, figsize=(14, 10))
axs = axs.flatten()

for split_i in range(total_splits):
    ax = axs[split_i]
    ax.plot(xgb_mae[split_i]["validation_1"]["mae"], label=f"XGB")
    ax.plot(lgbm_mae[split_i]["valid_1"]["l1"], label=f"LGBM")
    ax.plot(cb_mae[split_i]["validation_1"]["MAE"], label=f"CB")

    if split_i >= total_splits - cols:
        ax.set_xlabel("Number of estimators", size=9)
    if split_i in [i * cols for i in range(rows)]:
        ax.set_ylabel("MAE", size=9)
    ax.set_title(f"{split_i + 1} split", size=10)
    ax.grid(which="both", alpha=0.5)

plt.legend(
    loc="upper left",
    bbox_to_anchor=(1, 2.2),
    frameon=False,
)
fig.suptitle(
    "MAE for all models",
    y=0.925,
    fontsize=12,
)
# plt.tight_layout()
plt.show()

In [ ]:
# xgb_results = [
#     xgb_mae[i]["validation_1"]["mae"][-1] for i in range(len(xgb_mae))
# ]
# lgbm_results = [lgbm_mae[i]["valid_1"]["l1"][-1] for i in range(len(lgbm_mae))]
# cb_results = [
#     cb_mae[i]["validation_1"]["MAE"][-1] for i in range(len(lgbm_mae))
# ]

# print("xgb mae:", np.round(np.mean(xgb_results), 3))
# print("lgbm mae:", np.round(np.mean(lgbm_results), 3))
# print("cb mae:", np.round(np.mean(cb_results), 3))

In [ ]:
# for split in [[xgb_mae[i]["validation_1"]["mae"][-1] for i in range(split * 5, split * 5 + 5)] for split in range(4)]:
#     print(np.round(np.mean(split), 3))

In [ ]:
for split in [[lgbm_mae[i]["valid_1"]["l1"][-1] for i in range(split * 5, split * 5 + 5)] for split in range(4)]:
    print(np.round(np.mean(split), 3))

In [ ]:
for split in [[cb_mae[i]["validation_1"]["MAE"][-1] for i in range(split * 5, split * 5 + 5)] for split in range(4)]:
    print(np.round(np.mean(split), 3))

In [ ]:
[[i for i in range(split * 5, split * 5 + 5)] for split in range(4)]

In [ ]:
# residuals = y_test - y_pred

# plt.scatter(y_pred, residuals, alpha=0.5)
# plt.axhline(0, color="red", linestyle="--")
# plt.xlabel("Predicted")
# plt.ylabel("Residuals")
# plt.title("Residuals vs Predicted")
# plt.show()

In [ ]:
# fi = pd.Series(xgbr.feature_importances_, index=X_train.columns)
# fi = fi.sort_values(ascending=False).head(20)

# fi.plot.barh(figsize=(8,6))
# plt.xlabel("Feature Importance")
# plt.gca().invert_yaxis()
# plt.title("Top 20 Important Features")
# plt.show()